# <B> api-gateway with lambda </B>

## AutoReload

In [1]:
%load_ext autoreload
%autoreload 2

## 0. Get parameters 

In [2]:
import boto3
from utils.ssm import parameter_store

In [3]:
strRegionName=boto3.Session().region_name
pm = parameter_store(strRegionName)

## 1. Create lambda function

In [4]:
import shutil
import zipfile
import tempfile
from utils.lambda_func import lambda_handler

In [5]:
strPrefix = pm.get_params(key="PREFIX")
lam_handler = lambda_handler(region_name=pm.get_params(key="".join([strPrefix, "REGION"])))

In [6]:
strLambdaRoleArn = pm.get_params(key=strPrefix+"LAMBDA-ROLE-ARN")
strLambdaFuncName = strPrefix+"Lambda"
strLambdaSrcDir = "./lambda/source"
strLambdaHandler = "demo_lambda.lambda_handler" 

In [7]:
with tempfile.TemporaryDirectory() as tempDirPath:
    
    lambda_archive_path = shutil.make_archive(
        base_name=tempDirPath,
        format="zip",
        root_dir=strLambdaSrcDir,
    )
    
    with open(lambda_archive_path, 'rb') as f: zipped_code = f.read()
    
    strLambdaArn = lam_handler.create_function(
        Code=dict(ZipFile=zipped_code),
        Description='Demo: api gateway with lambda',
        Environment={
           'Variables': {
               'REGION':pm.get_params(key=strPrefix+"REGION"),
               'INFERENCE_RES_TABLE': pm.get_params(key=strPrefix+"INFERENCE-RES-TABLE-NAME"),
           },
        },
        FunctionName=strLambdaFuncName,
        Handler=strLambdaHandler,
        Publish=True,
        Role=strLambdaRoleArn,
        Runtime='python3.9',
    )
    
print (f"LambdaArn: {strLambdaArn}")
pm.put_params(key=strPrefix+"LAMBDA-NAME", value=strLambdaFuncName, overwrite=True)

== CREATE LAMBDA FUNCTION ==
  lambda function: [API-GATEWAY-LAMBDA-DEMO-Lambda] is already exist!!, so, this will be deleted and re-created.
  lambda function: [API-GATEWAY-LAMBDA-DEMO-Lambda] is deleted successfully
Argments for lambda below:

{'Architectures': ['x86_64'],
 'CodeSha256': '2APWn5xf6AniyOwYUHu/jfFooHEuC8ehZMLNrKey7Hg=',
 'CodeSize': 2096,
 'Description': 'Demo: api gateway with lambda',
 'Environment': {'Variables': {'INFERENCE_RES_TABLE': 'API-GATEWAY-LAMBDA-DEMO-inference-results',
                               'REGION': 'ap-northeast-2'}},
 'EphemeralStorage': {'Size': 512},
 'FunctionArn': 'arn:aws:lambda:ap-northeast-2:419974056037:function:API-GATEWAY-LAMBDA-DEMO-Lambda',
 'FunctionName': 'API-GATEWAY-LAMBDA-DEMO-Lambda',
 'Handler': 'demo_lambda.lambda_handler',
 'LastModified': '2022-12-09T02:31:17.519+0000',
 'MemorySize': 128,
 'PackageType': 'Zip',
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'conte

'Store suceess'

## 2. Create API-Gateway with Lambda

In [8]:
from utils.api_gateway import api_gateway_handler

In [9]:
strAccountId = pm.get_params(key=strPrefix+"ACCOUNT-ID")
strApiName = strPrefix+"api-gateway"
strApiDesc = "api-gatway for demo"
strApiBasePath = "demo-api"
strApiStage = "dev"
strLambdaFuncArn = strLambdaArn
apig = api_gateway_handler(pm.get_params(key=strPrefix+"REGION"))

In [10]:
strLambdaFuncArn

'arn:aws:lambda:ap-northeast-2:419974056037:function:API-GATEWAY-LAMBDA-DEMO-Lambda'

In [12]:
strApiUrl, strApiId = apig.create_rest_api_with_lambda(
    api_name=strApiName,
    api_description=strApiDesc,
    api_base_path=strApiBasePath,
    api_stage=strApiStage,
    account_id=strAccountId,
    lambda_function_arn=strLambdaFuncArn
)
pm.put_params(key=strPrefix+"API-URL", value=strApiUrl.encode('utf-8'), overwrite=True)
pm.put_params(key=strPrefix+"API-ID", value=strApiId, overwrite=True)

Creating Amazon API Gateway REST API API-GATEWAY-LAMBDA-DEMO-api-gateway...
so arn:aws:execute-api:ap-northeast-2:419974056037:5q95fu1g5d/*/*/demo-api
REST API created, URL is :
	https://5q95fu1g5d.execute-api.ap-northeast-2.amazonaws.com/dev/demo-api
Sleeping for a couple seconds to give AWS time to prepare...


'Store suceess'

## 3. Test

In [12]:
import json
import requests
from pprint import pprint 

In [13]:
strApiUrl

'https://t77j63kgch.execute-api.ap-northeast-2.amazonaws.com/dev/demo-api'

In [14]:
dicRequest = {
    "userCode": "106561002",
    "productInfo":{
        "productCode": ["373120", "291221", "297909", "359769", "279150", "259401", "520967"],
    },
    "option": {
        "manualTopn": "10",
        "bestTopn": "10",
        "crossTopn": "10"
    },
}

try:
    response = requests.post(strApiUrl, json.dumps(dicRequest))
    response = json.loads(response.content)
    #results = response["resultData"]

    pprint (response)
    
except Exception as e:
    print ("ERROR", e)

('{"resultCode": "200", "resultMsg": "Success", "resultData": [{"productCode": '
 '"325396", "recommendType": "best", "rank": "1"}, {"productCode": "122762", '
 '"recommendType": "best", "rank": "2"}, {"productCode": "122751", '
 '"recommendType": "best", "rank": "3"}, {"productCode": "359769", '
 '"recommendType": "best", "rank": "4"}, {"productCode": "338779", '
 '"recommendType": "best", "rank": "5"}, {"productCode": "168337", '
 '"recommendType": "best", "rank": "6"}, {"productCode": "165245", '
 '"recommendType": "best", "rank": "7"}, {"productCode": "134571", '
 '"recommendType": "best", "rank": "8"}, {"productCode": "281558", '
 '"recommendType": "best", "rank": "9"}, {"productCode": "514939", '
 '"recommendType": "best", "rank": "10"}, {"productCode": "278158", '
 '"recommendType": "cross", "rank": "1"}, {"productCode": "101583", '
 '"recommendType": "cross", "rank": "2"}, {"productCode": "159994", '
 '"recommendType": "cross", "rank": "3"}, {"productCode": "252430", '
 '"recom